In [1]:
import numpy as np
import pandas as pd

orders = 'instacart_2017_05_01/orders.csv'
products = 'instacart_2017_05_01/products.csv'
prior = 'instacart_2017_05_01/order_products__prior.csv'
train = 'instacart_2017_05_01/order_products__train.csv'
aisles = 'instacart_2017_05_01/aisles.csv'
departments = 'instacart_2017_05_01/departments.csv'


In [2]:
df_orders = pd.read_csv(orders)
df_orders
Number_orders = len(df_orders)

In [3]:
df_orderProductsTrain = pd.read_csv(train)

In [4]:
df_orderProductsPrior = pd.read_csv(prior)


In [5]:
(len(df_orderProductsPrior) + len(df_orderProductsTrain))/len(df_orders)

9.885497077972092

In [6]:
df_aisles= pd.read_csv(aisles)

In [7]:
df_products= pd.read_csv(products)

In [8]:
df_departments= pd.read_csv(departments)

ideas:
- look for rules on 3 levels: product (big data challenge), aile, department (algorithm challenge)
- see how this changes accross time


In [9]:
df_allOrdersProducts = pd.concat([df_orderProductsPrior, df_orderProductsTrain], axis = 0)

In [10]:
df_allOrdersProducts = df_allOrdersProducts.reset_index()

### Frequent itemset generation

In [21]:
df_countProductsFrequency = df_allOrdersProducts.groupby(['product_id']).agg({'order_id': lambda x: len(tuple(x)) / Number_orders})

In [22]:
df_countProductsFrequency = df_countProductsFrequency.reset_index()
df_countProductsFrequency

,product_id,order_id
0,1,0.000564
1,2,0.000027
2,3,0.000083
3,4,0.000103
4,5,0.000005
5,6,0.000002
6,7,0.000009
7,8,0.000052
8,9,0.000047
9,10,0.000787
